In [ ]:
from pydantic import BaseModel

class VideoInput(BaseModel):
    name: str
    description: str
    url: str
    duration: int
    output_folder: str = "output"

class DetectionInput(BaseModel):
    name: str
    description: str
    video_id: str
    region_ids: list[str]
    model_name: str = "yolo11n.pt"
    tracker: str = "botsort.yaml"
    classes: list[int] = [0]
    max_frames: int = -1
    save: True

class RenderInput(BaseModel):
    video_id: str
    detection_id: str

In [ ]:
import os
import uuid
import json
import subprocess
from datetime import datetime

class VideoInput(BaseModel):
    name: str
    description: str
    url: str
    duration: int
    output_folder: str = "output"

def download_video(video_input: VideoInput):
    """
    Menyimpan video stream dari URL (.m3u8) selama durasi tertentu ke output_folder/name
    dan menyimpan metadata dalam format JSON.

    Args:
        name (str): Nama file output (tanpa ekstensi).
        description (str): Deskripsi CCTV.
        url (str): URL stream (m3u8).
        duration (int): Durasi rekaman dalam detik.
        output_folder (str): Folder output utama.
    """
    video_id = str(uuid.uuid4())
    output_folder = os.path.join(output_folder, video_id)
    os.makedirs(output_folder, exist_ok=True)

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_filename = f"{video_input.name}_{timestamp}.mp4"
    output_path = os.path.join(output_folder, output_filename)

    ffmpeg_cmd = [
        'ffmpeg',
        '-y',
        '-i', video_input.url,
        '-t', str(video_input.duration),
        '-c', 'copy',
        output_path
    ]

    print(' '.join(ffmpeg_cmd))

    try:
        print(f"[INFO] Saving stream to {output_path} for {duration} seconds...")
        subprocess.run(ffmpeg_cmd, check=True)
        print(f"[DONE] Video saved: {output_path}")

        # Simpan metadata
        metadata = {
            "id": video_id,
            "name": name,
            "description": description,
            "url": url,
            "duration": duration,
            "created_at": datetime.now().isoformat(),
            "video_path": output_path
        }

        metadata_path = os.path.join(output_folder, "metadata.json")
        with open(metadata_path, "w", encoding="utf-8") as f:
            json.dump(metadata, f, indent=2)
        
        print(f"[INFO] Metadata saved to {metadata_path}")

    except subprocess.CalledProcessError as e:
        print(f"[ERROR] Gagal menyimpan stream: {e}")


In [ ]:
import os
import uuid
import cv2
import json
import numpy as np
from datetime import datetime, timedelta
from ultralytics import YOLO
from pydantic import BaseModel
from typing import Optional, Any
from db import PostgresDatabase

class DetectionInput(BaseModel):
    name: str
    description: str
    video_id: str
    region_ids: list[str]
    model_name: str = "yolo11n.pt"
    tracker: str = "botsort.yaml"
    classes: list[int] = [0]
    max_frames: int = -1
    save: bool = True

def detect_video(detection_input: DetectionInput, db: PostgresDatabase):
    # Ambil metadata video
    video_info = db.execute_query("SELECT video_path, output_folder, frame_rate, video_start_timestamp FROM video_registry WHERE video_id = %s", (detection_input.video_id,))
    if not video_info:
        raise ValueError("Video not found")
    video_path, output_folder, frame_rate, video_start_timestamp = video_info[0]

    # Ambil data region
    placeholders = ','.join(['%s'] * len(detection_input.region_ids))
    query = f"SELECT region_id, region_name, region_description, region_polygon FROM region_registry WHERE region_id IN ({placeholders})"
    region_rows = db.execute_query(query, tuple(detection_input.region_ids))
    region_definitions = [
        {
            "id": str(row[0]),
            "name": row[1],
            "description": row[2],
            "polygon": row[3]  # diasumsikan list of (x, y)
        }
        for row in region_rows
    ]

    # Insert job
    detection_id = str(uuid.uuid4())
    db.execute_query(
        """
        INSERT INTO detection_jobs (
            pk_detection_id, fk_video_id, name, description, classes,
            model_name, tracker, max_frame, created_at
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, NOW())
        """,
        (
            detection_id,
            detection_input.video_id,
            detection_input.name,
            detection_input.description,
            json.dumps(detection_input.classes),
            detection_input.model_name,
            detection_input.tracker,
            detection_input.max_frames
        )
    )

    # Mapping region ke job
    for region in detection_input.region_ids:
        db.execute_query(
            "INSERT INTO detect_region_mapping (fk_detection_id, fk_region_id) VALUES (%s, %s)",
            (detection_id, region)
        )

    # Video processing
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"

    model = YOLO(detection_input.model_name)
    class_names = model.names
    fps = frame_rate or cap.get(cv2.CAP_PROP_FPS) or 30

    frame_number = 0
    video_start_dt = datetime.fromisoformat(video_start_timestamp.replace("Z", "+00:00"))
    video_filename = os.path.basename(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or (detection_input.max_frames > 0 and frame_number >= detection_input.max_frames):
            break

        current_timestamp = video_start_dt + timedelta(seconds=frame_number / fps)
        result = model.track(frame, persist=True, conf=0.25, verbose=False, tracker=detection_input.tracker, classes=detection_input.classes)[0]

        for region in region_definitions:
            count = 0
            frame_objects = []

            for i, box in enumerate(result.boxes):
                cls_id = int(box.cls.item())
                cls_name = class_names[cls_id]
                if cls_id not in detection_input.classes:
                    continue

                bbox = box.xyxy[i].tolist()
                track_id = int(box.id.item()) if box.id is not None else None
                conf = float(box.conf.item())
                cx = (bbox[0] + bbox[2]) / 2
                cy = (bbox[1] + bbox[3]) / 2

                inside = cv2.pointPolygonTest(np.array(region['polygon'], dtype=np.int32), (cx, cy), False) >= 0
                if inside:
                    count += 1

                frame_objects.append({
                    "tracker_id": track_id,
                    "bbox": bbox,
                    "confidence": conf,
                    "inside_region": inside
                })

            # Simpan ke detection_event
            db.execute_query(
                """
                INSERT INTO detection_event (
                    fk_detection_id, fk_region_id, frame_number, timestamp, count, created_at
                ) VALUES (%s, %s, %s, %s, %s, NOW()) RETURNING pk_detection_event_id
                """,
                (
                    detection_id, region['id'], frame_number, current_timestamp.isoformat() + 'Z', count
                )
            )
            event_id = db.execute_query("SELECT LASTVAL()")
            event_id = event_id[0][0]

            for obj in frame_objects:
                db.execute_query(
                    """
                    INSERT INTO detection_objects (
                        fk_detection_event_id, tracker_id, bbox, confidence, inside_region, created_at
                    ) VALUES (%s, %s, %s, %s, %s, NOW())
                    """,
                    (
                        event_id,
                        obj["tracker_id"],
                        json.dumps(obj["bbox"]),
                        obj["confidence"],
                        obj["inside_region"]
                    )
                )

        frame_number += 1

    cap.release()
    print(f"Detection job {detection_id} selesai.")
    return detection_id

# Contoh UUID dari database Anda
VIDEO_ID = "750775f1-6962-4388-9c13-e6d5f2c2c327"
REGION_IDS = [
    "d75cf6e4-851e-47f5-9d65-0a9304c03801",
    "2018db39-324a-460b-a3f5-2850978bd700"
]

detection_input = DetectionInput(
    name="Deteksi Orang di Halte",
    description="Uji coba deteksi orang dengan YOLOv8 dan BotSort",
    video_id=VIDEO_ID,
    region_ids=REGION_IDS,
    model_name="yolov8n.pt",
    tracker="botsort.yaml",
    classes=[0],  # hanya 'person'
    max_frames=100,
    save=True
)

results = detect_video(detection_input)
print(f"Jumlah hasil deteksi: {len(results)}")
print("Contoh hasil deteksi pertama:")
if results:
    from pprint import pprint
    pprint(results[0])
